In [ ]:
from email.parser import Parser
import re
import time
from datetime import datetime, timezone, timedelta

pretty = lambda x: '\n'.join(str(e) for e in x) if x else None
pretty_rdd = lambda x: pretty(x.collect()) if x else None

def date_to_dt(date):
    def to_dt(tms):
        def tz():
            return timezone(timedelta(seconds=tms.tm_gmtoff))
        return datetime(tms.tm_year, tms.tm_mon, tms.tm_mday, 
                      tms.tm_hour, tms.tm_min, tms.tm_sec, 
                      tzinfo=tz())
    return to_dt(time.strptime(date[:-6], '%a, %d %b %Y %H:%M:%S %z'))

# Helper function to decode
# the emails from the input dataset
# and ignore malformated entries 
def utf8_decode_and_filter(rdd):
    def utf_decode(s):
        try:
          return str(s, 'utf-8')
        except:
            pass
    return rdd.map(lambda x: utf_decode(x[1])).filter(lambda x: x != None)

## Q1)

In [ ]:
def extract_email_network(rdd):
    import re
    val_by_vec=lambda x,t,p:((x,item,p) for item in t)
    email_regex = "[\w.!#$%&'*+-/=?^_`{|}~]+@([a-zA-Z0-9]*\.+(enron.com)$|(enron.com)$)"
    valid_email=(lambda s: True if re.compile(email_regex).fullmatch(s)!=None else False)
    
    rdd_mail=rdd.map(lambda item:Parser().parsestr(item))
    rdd_full_email_tuples=rdd_mail.map(lambda item:(item.get('From'),[item.get('To'), item.get('Cc'), item.get('Bcc')],date_to_dt(item.get('Date'))))
    rdd_email_triples=rdd_full_email_tuples.flatMap(lambda x:val_by_vec(x[0],x[1],x[2]))
    rdd_email_triples_enron=rdd_email_triples.filter(lambda x: x[1] != None)\
    .filter(lambda x:x[0]!=x[1])\
    .map(lambda x:(x[0],re.sub('\,*\s+',',',x[1]),x[2]))\
    .map(lambda x:(x[0],x[1].split(','),x[2]))\
    .flatMap(lambda x:val_by_vec(x[0],x[1],x[2]))\
    .map(lambda x:(x[0],x[1],x[2]))\
    .filter(lambda x:valid_email(x[0]) and valid_email(x[1]))
    distinct_triples = rdd_email_triples_enron.distinct()
    
    return(distinct_triples)

## Q2)

In [ ]:
def convert_to_weighted_network(rdd,drange=None):
    if drange!=None:
        rdd_result=rdd.filter(lambda x:drange[1]>=x[2] and x[2]>=drange[0]).map(lambda x:((x[0],x[1]),1))\
        .reduceByKey(lambda x,y:x+y).map(lambda x:(x[0][0],x[0][1],x[1])).sortBy(lambda x:x[1],False).sortBy(lambda x:x[0])
        return(rdd_result)
    elif drange==None:
        rdd_result=rdd.map(lambda x:((x[0],x[1]),1)).reduceByKey(lambda a,x:a+x)\
        .map(lambda x:(x[0][0],x[0][1],x[1])).sortBy(lambda x:x[0])
        return(rdd_result)

In [ ]:
rdd = extract_email_network(
        utf8_decode_and_filter(sc.sequenceFile(
                '/user/uxac007/project1819/samples/enron20.seq')))
(lambda rdd: rdd.cache() if rdd else None)(rdd)

## Q3.1)

In [ ]:
def get_out_degrees(rdd):
    rdd_sender=rdd.map(lambda x:(x[0],x[2]))\
    .reduceByKey(lambda a,x:a+x).map(lambda x:(x[1],x[0])).sortBy(lambda x: x[1],False).sortBy(lambda x: x[0], False)
    
    rdd_receiver=rdd.map(lambda x:x[1]).subtract(rdd.keys()).distinct().map(lambda x:(0,x)).sortBy(lambda x: x[1],False)
    
    rdd_result=rdd_sender.union(rdd_receiver)
    
    return(rdd_result)

## Q3.2)

In [ ]:
def get_in_degrees(rdd):
    rdd_sender=rdd.map(lambda x:(x[1],x[2])).reduceByKey(lambda a,x:a+x)\
    .map(lambda x:(x[1],x[0])).sortBy(lambda x:x[1],False).sortBy(lambda x:x[0],False)
    
    rdd_receiver=rdd.map(lambda x:x[0]).subtract(rdd.map(lambda x:x[1]))\
    .distinct().map(lambda x:(0,x)).sortBy(lambda x:x[1],False)
    
    return(rdd_sender.union(rdd_receiver))

## Q4.1)

In [ ]:
def get_out_degree_dist(rdd):
    rdd_sender=rdd.map(lambda x:(x[0],x[2])).reduceByKey(lambda a,x:a+x).map(lambda x:(x[1],x[0]))
    
    rdd_receiver=rdd.map(lambda x:x[1]).subtract(rdd.keys()).distinct().map(lambda x:(0,x))
    
    rdd_result=rdd_sender.union(rdd_receiver).map(lambda x:(x[0],1))\
    .reduceByKey(lambda a,x:a+x).sortBy(lambda x:x[0])
    
    return(rdd_result)

## Q4.2)

In [ ]:
def get_in_degree_dist(rdd):
    rdd_receiver=pp.map(lambda x:(x[1],x[2])).reduceByKey(lambda a,x:a+x).map(lambda x:(x[1],x[0]))
    
    rdd_sender=pp.map(lambda x:x[0]).subtract(pp.map(lambda x:x[1])).distinct().map(lambda x:(0,x,1))
    
    rdd_result=rdd_sender.union(rdd_receiver).map(lambda x:(x[0],1))\
    .reduceByKey(lambda a,x:a+x).sortBy(lambda x:x[0])
    
    return(rdd_result)